In [1]:
import pandas as pd
import numpy as np
import pyreadstat  ##conda install conda-forge::pyreadstat
import sqlalchemy as sa
from sqlalchemy.engine import URL
import pyodbc

In [3]:
server = 'gisvaltech2025.cucrxmmsbhhf.us-east-1.rds.amazonaws.com' 
database = 'workshop' 
username = 'tester' 
password = 'gisvaltech' 

connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s;DATABASE=%s;UID=%s;PWD=%s" % (server, database, username, password)

In [15]:
## Read SPSS sav file using read_spss

data = pd.read_spss('..\\SPSS\\Data\\GISValTechSampleData.sav')
data.head()
data['ParcelId'] = data['ParcelId'].astype(int)
data['SaleDate'] = pd.to_datetime(data['SaleDate'])
data['SalesPrice'] = data['SalesPrice'].astype(int)
data['Sqft'] = data['Sqft'].astype(int)
data['LandSize'] = data['LandSize'].astype(int)
data['Quality'] = pd.Categorical(data['Quality'], ordered = True, categories = [
    'Poor', 'BelowAverage', 'Average', 'AboveAverage', 'Superior'])
data['GarageSize'] = data['GarageSize'].astype(int)
data['EffAge'] = data['EffAge'].astype(int)
data['NBHD'] = pd.Categorical(data['NBHD'].astype(int))

data.set_index('ParcelId', inplace = True)
data.head()

,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
ParcelId,,,,,,,,,
103525,2023-01-01,254596,1573,27325,2.5,BelowAverage,360,41,102
110784,2023-01-01,1151736,4586,14157,5.0,AboveAverage,484,18,105
116454,2023-01-01,501824,2548,17156,3.5,Average,384,10,103
138782,2023-01-02,789205,3312,34341,2.5,AboveAverage,360,16,104
102652,2023-01-03,548042,2455,34412,2.5,AboveAverage,484,55,104


In [ ]:
# Export data to spss using pyreadstat
pyreadstat_data = pd.DataFrame({"ParcelId" : [12345, 12346, 12347, 12348],
                     "LivingArea" : [1500, 1800, 2000, 2250]})

pyreadstat.write_sav(pyreadstat_data, '..//exported data//spss.sav')

In [ ]:
## Extract data from SQL database using pyodbc
## without a warning

cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
cursor.execute("""SELECT [ParcelId], [SaleDate], [SalesPrice], [Sqft], [LandSize], [Bathrooms], [Quality],
      [GarageSize], [EffAge], [NBHD] FROM SampleData""")
pyodbc_data = pd.DataFrame([])
for row in cursor.fetchall():
    data = pd.Series([elem for elem in row])
    pyodbc_data = pd.concat([pyodbc_data, data.to_frame().T], ignore_index=True)

pyodbc_data.columns = ["ParcelId", "SaleDate", "SalesPrice", "Sqft", "LandSize", "Bathrooms", "Quality",
      "GarageSize", "EffAge", "NBHD"]

print(pyodbc_data)



    ParcelId    SaleDate SalesPrice  Sqft LandSize Bathrooms       Quality  \
0     103525  2023-01-01     254596  1573    27325       2.5  BelowAverage   
1     110784  2023-01-01    1151736  4586    14157       5.0  AboveAverage   
2     116454  2023-01-01     501824  2548    17156       3.5       Average   
3     138782  2023-01-02     789205  3312    34341       2.5  AboveAverage   
4     102652  2023-01-03     548042  2455    34412       2.5  AboveAverage   
..       ...         ...        ...   ...      ...       ...           ...   
817   111457  2023-12-27     197658  1576    20188       2.5          Poor   
818   109969  2023-12-28     322208  1493    22137       3.0  BelowAverage   
819   113733  2023-12-29     144949  1148    15213       2.0  BelowAverage   
820   105513  2023-12-30     306441  1551    13181       3.5  BelowAverage   
821   123667  2023-12-31     933604  3192    39304       3.5  AboveAverage   

    GarageSize EffAge NBHD  
0          360     41  102  
1    

In [6]:
## Extract data from SQL database using pyodbc
## this will throw a warning

cnxn = pyodbc.connect(connection_string)
data = pd.read_sql_query("SELECT * FROM SampleData", cnxn)
data

C:\Users\Brad\AppData\Local\Temp\ipykernel_23204\2994711458.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query("SELECT * FROM SampleData", cnxn)


,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
0,103525,2023-01-01,254596,1573,27325,2.5,BelowAverage,360,41,102
1,110784,2023-01-01,1151736,4586,14157,5.0,AboveAverage,484,18,105
2,116454,2023-01-01,501824,2548,17156,3.5,Average,384,10,103
3,138782,2023-01-02,789205,3312,34341,2.5,AboveAverage,360,16,104
4,102652,2023-01-03,548042,2455,34412,2.5,AboveAverage,484,55,104
...,...,...,...,...,...,...,...,...,...,...
817,111457,2023-12-27,197658,1576,20188,2.5,Poor,360,58,102
818,109969,2023-12-28,322208,1493,22137,3.0,BelowAverage,264,29,102
819,113733,2023-12-29,144949,1148,15213,2.0,BelowAverage,0,73,101
820,105513,2023-12-30,306441,1551,13181,3.5,BelowAverage,0,31,102


In [9]:
## Export data to SQL using pyodbc
cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
pyodbc_data = pd.DataFrame({
    "ParcelId" : [12345, 12346, 12347, 12348],
    "SaleDate" : ['01-01-2023', '02-01-2023', '03-01-2023', '04-01-2023'],
    "SalesPrice" : [100000, 150000, 200000, 250000]
})
for index, row in pyodbc_data.iterrows():
    cursor.execute("INSERT INTO pyodbcTestData (ParcelId, SaleDate, SalesPrice) values (?, ?, ?)", row['ParcelId'], row['SaleDate'], row['SalesPrice'])
cnxn.commit()
cursor.close()

In [10]:
## Extract data from SQL database using sqlalchemy

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

with engine.begin() as conn:
   sqlalchemy_data = pd.read_sql_query(sa.text("SELECT * FROM SampleData;"), conn)

sqlalchemy_data.sample(5)

,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
580,101168,2023-09-09,330481,1828,16870,3.5,Average,360,42,103
804,139521,2023-12-16,300893,1793,18659,2.0,BelowAverage,360,52,102
550,108682,2023-08-28,534220,1941,15273,2.5,Superior,384,45,103
702,135117,2023-11-05,244915,1340,13792,2.5,BelowAverage,384,10,102
735,140320,2023-11-14,354971,1905,15128,2.0,Average,360,47,102


In [11]:
## Export data to SQL using sqlalchemy
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

sqlalchemy_data = pd.DataFrame({
    "ParcelId" : [12345, 12346, 12347, 12348],
    "EffAge" : [12, 23, 44, 67]
})
sqlalchemy_data.to_sql(name='TestData_Brad', if_exists='replace', con=engine)

4

In [16]:
## Export data to a pickle file
data.to_pickle('..\\exported data\\PickleExport.pkl')

In [17]:
## Read pickle file

data = pd.read_pickle('..\\exported data\\PickleExport.pkl')
data.sample(5)

,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
ParcelId,,,,,,,,,
118195,2023-08-13,397065,1856,10164,3.5,AboveAverage,360,11,103
136360,2023-11-23,372980,2084,21371,3.5,AboveAverage,360,32,104
130525,2023-06-04,157510,1462,14849,2.5,BelowAverage,0,79,101
107836,2023-10-28,1178513,3547,10967,4.0,Superior,360,3,105
101425,2023-05-04,505123,2723,12787,3.0,Average,264,7,103


In [19]:
## Export to excel file
data.to_excel('..\\exported data\\excelExport.xlsx')

In [20]:
## Read excel file

data = pd.read_excel('..\\exported data\\excelExport.xlsx')
data.sample(5)

,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
390,110471,2023-06-26,214667,1369,17091,3.0,BelowAverage,360,42,102
753,104147,2023-11-23,174727,1307,12955,2.5,Average,0,42,101
515,118195,2023-08-13,397065,1856,10164,3.5,AboveAverage,360,11,103
583,117596,2023-09-11,1340291,4494,19389,4.0,Superior,484,4,105
580,101168,2023-09-09,330481,1828,16870,3.5,Average,360,42,103


In [21]:
## Export data to csv
data.to_csv('..\\exported data\\CsvExport.csv')

In [22]:
## Read csv file
data = pd.read_csv('..\\exported data\\CsvExport.csv')
data.sample(5)

,Unnamed: 0,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
513,513,105209,2023-08-12,265864,1465,19242,3.5,BelowAverage,384,37,102
85,85,130628,2023-01-21,377974,1755,37433,2.5,Average,704,31,104
45,45,119941,2023-01-13,1116042,3838,16256,5.5,AboveAverage,704,10,105
127,127,137252,2023-02-07,207578,1111,11554,3.0,BelowAverage,384,22,102
104,104,133249,2023-01-27,268237,1398,15001,2.5,AboveAverage,0,20,102


195